In [98]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
import sys
from sklearn.metrics import accuracy_score
from collections import Counter
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn import linear_model
from sklearn.linear_model import Perceptron
from sklearn.neighbors import KNeighborsClassifier

In [2]:
df = pd.read_csv('CleanIMDB_train.csv') # Windows single-byte character encoding
df.head()

,review,sentiment
0,steve carell stars person relate to sort of da...,1
1,found enjoyable muppets movie felt light heart...,1
2,altogether bad start program slap face real la...,0
3,saw film tonight nyc landmark sunshine did not...,1
4,much like japanese movies one did not cut it m...,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 2 columns):
review       5000 non-null object
sentiment    5000 non-null int64
dtypes: int64(1), object(1)
memory usage: 78.2+ KB


In [4]:
print('Total dataset length: %d' %(len(df)))

Total dataset length: 5000


In [5]:
# Split the data to train and test
X = df.iloc[:, 0].values
y = df.iloc[:, 1].values

In [6]:
cvec = CountVectorizer()
# vectorize the data set
X_matrix = cvec.fit_transform(X)
print('Train data count vector total number of features: %d' %(len(cvec.get_feature_names())))
X_matrix.shape
X_matrix_ = X_matrix.toarray()

Train data count vector total number of features: 37996


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_matrix_,y,test_size=0.20, random_state=42)
y_test.shape

(1000,)

In [8]:
print('Total training entries length: %d' %(len(X_train)))
print('Negative training entries account for percent: %f' %(Counter(y_train)[0]/len(y_train)*100))
print('Positive training entries account for percent: %f' %(Counter(y_train)[0]/len(y_train)*100))
print('Total test entries length: %d' %(len(y_test)))
print('Negative test entries account for percent: %f' %(Counter(y_test)[0]/len(y_test)*100))
print('Positive test entries account for percent: %f' %(Counter(y_test)[1]/len(y_test)*100))

Total training entries length: 4000
Negative training entries account for percent: 51.125000
Positive training entries account for percent: 51.125000
Total test entries length: 1000
Negative test entries account for percent: 51.900000
Positive test entries account for percent: 48.100000


In [9]:
# Split the train to train and validation - Holdout Method
X_train_, X_val,y_train_,y_val = train_test_split(X_train,y_train,test_size=0.20, random_state=42)

In [10]:
print("Total training entries after validation split: %d" %(len(X_train_)))
print('Negative train entries after val split account for percent: %f' %(Counter(y_train_)[0]/len(y_train_)*100))
print('Positive train entries after val split account for percent: %f' %(Counter(y_train_)[1]/len(y_train_)*100))
print("Total validation entries after train split: %d" %(len(X_val)))
print('Negative validation entries after val split account for percent: %f' %(Counter(y_val)[0]/len(y_val)*100))
print('Positive validation entries after val split account for percent: %f' %(Counter(y_val)[1]/len(y_val)*100))

Total training entries after validation split: 3200
Negative train entries after val split account for percent: 51.156250
Positive train entries after val split account for percent: 48.843750
Total validation entries after train split: 800
Negative validation entries after val split account for percent: 51.000000
Positive validation entries after val split account for percent: 49.000000


In [26]:
# Baseline
log_model = LogisticRegression(C=1.0, random_state=None, solver='liblinear', penalty='l2')
# Fit the train dataset with LR
log_model = log_model.fit(X=X_train_, y=y_train_)

In [27]:
y_pred = log_model.predict(X_val)
print('Accuracy validation dataset logistics regression: %f' % accuracy_score(y_val, y_pred))

Accuracy validation dataset logistics regression: 0.843750


In [13]:
confusion_matrix(y_val, y_pred)

array([[344,  64],
       [ 61, 331]], dtype=int64)

In [14]:
print(classification_report(y_val, y_pred))

             precision    recall  f1-score   support

          0       0.85      0.84      0.85       408
          1       0.84      0.84      0.84       392

avg / total       0.84      0.84      0.84       800



In [18]:
svm_model = SVC(C=100.0, kernel='rbf', random_state=None, gamma='auto')
svm_model = svm_model.fit(X_train_, y_train_)

In [19]:
y_pred = svm_model.predict(X_val)
print('Accuracy validation dataset SVM: %f' % accuracy_score(y_val, y_pred))

Accuracy validation dataset SVM: 0.841250


In [35]:
confusion_matrix(y_val, y_pred)

array([[408,   0],
       [ 23, 369]], dtype=int64)

In [36]:
print(classification_report(y_val, y_pred))

             precision    recall  f1-score   support

          0       0.95      1.00      0.97       408
          1       1.00      0.94      0.97       392

avg / total       0.97      0.97      0.97       800



In [75]:
# Naive Bayes Bernoulli
bnb = BernoulliNB(binarize=0.0)
bnb.fit(X_train_, y_train_)
print('Naive Bayes Bernoulli')
bnb.score(X_val, y_val)

Naive Bayes Bernoulli


0.815

In [96]:
# Multinominal Naive Bayes
mnb = MultinomialNB()
mnb.fit(X_train_, y_train_)
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
y_pred = mnb.predict(X_val)
print('Accuracy validation dataset Naive Bayes(Multinomonal): %f' % accuracy_score(y_val, y_pred))

Accuracy validation dataset Naive Bayes(Multinomonal): 0.841250


In [77]:
confusion_matrix(y_val, y_pred)

array([[363,  45],
       [ 82, 310]], dtype=int64)

In [78]:
print(classification_report(y_val, y_pred))

             precision    recall  f1-score   support

          0       0.82      0.89      0.85       408
          1       0.87      0.79      0.83       392

avg / total       0.84      0.84      0.84       800



In [49]:
clf = linear_model.SGDClassifier(max_iter=1000, tol=1e-3)
clf.fit(X_train_, y_train_)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=1000, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=0.001, verbose=0, warm_start=False)

In [50]:
y_pred = clf.predict(X_val)

In [51]:
print('Accuracy validation dataset AdalineSDG: %f' % accuracy_score(y_val, y_pred))

Accuracy validation dataset AdalineSDG: 0.825000


In [53]:
confusion_matrix(y_val, y_pred)

array([[350,  58],
       [ 82, 310]], dtype=int64)

In [54]:
print(classification_report(y_val, y_pred))

             precision    recall  f1-score   support

          0       0.81      0.86      0.83       408
          1       0.84      0.79      0.82       392

avg / total       0.83      0.82      0.82       800



In [55]:
clf = Perceptron(tol=1e-3, random_state=0)
clf.fit(X_train_, y_train_)

Perceptron(alpha=0.0001, class_weight=None, eta0=1.0, fit_intercept=True,
      max_iter=None, n_iter=None, n_jobs=1, penalty=None, random_state=0,
      shuffle=True, tol=0.001, verbose=0, warm_start=False)

In [56]:
y_pred = clf.predict(X_val)

In [57]:
print('Accuracy validation dataset Perceptron: %f' % accuracy_score(y_val, y_pred))

Accuracy validation dataset Perceptron: 0.836250


In [58]:
confusion_matrix(y_val, y_pred)

array([[344,  64],
       [ 67, 325]], dtype=int64)

In [59]:
print(classification_report(y_val, y_pred))

             precision    recall  f1-score   support

          0       0.84      0.84      0.84       408
          1       0.84      0.83      0.83       392

avg / total       0.84      0.84      0.84       800



In [102]:
knn = KNeighborsClassifier(n_neighbors = 5)
knn.fit(X_train_,y_train_)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [103]:
knn.predict(X_val)[0:5]

array([0, 1, 1, 1, 0], dtype=int64)

In [104]:
knn.score(X_test, y_test)

0.585

In [105]:
y_pred = log_model.predict(X_test)
print('Accuracy test dataset logistics regression: %f' % accuracy_score(y_test, y_pred))

Accuracy test dataset logistics regression: 0.855000
